In [2]:
import numpy as np
import sys
import itertools


# Initialize parameters
L = 5  # Lattice size
N_a = 10 # Number of lattice points along each direction
N = N_a**3  # Total number of lattice
T = 1.0  # Temperature
sample_size = int(5e3)  # Total number of Monte Carlo steps

cutoff = 3.0  # Cutoff distance for LJ potential
epsilon = 1.0  # Depth of the potential well
sigma = 1.0  # Length scale in LJ potential, also the distance at which the potential becomes zero

# Lennard-Jones potential function
def V_ij(ri, rj, epsilon=epsilon, sigma=sigma, cutoff=cutoff,uv_cutoff=0.90):
    r_ij = np.linalg.norm(np.array(ri) - np.array(rj))
    if r_ij > cutoff:
        return 0.0
    if r_ij < uv_cutoff:
        r_ij = uv_cutoff
    return 4 * epsilon * ((sigma / r_ij)**12 - (sigma / r_ij)**6)

def compute_h_field(lattice, L=L, N_a=N_a, cutoff=cutoff):
    h_field = np.zeros_like(lattice, dtype=float)
    a = L / (N_a-1)  # Lattice constant of the underlying simple cubic lattice (in reduced units) 
    for i, j, k in itertools.product(range(N_a), range(N_a), range(N_a)):
        count = 0
        for dx in range(-int(cutoff // a), int(cutoff // a) + 1):
            for dy in range(-int(cutoff // a), int(cutoff // a) + 1):
                for dz in range(-int(cutoff // a), int(cutoff // a) + 1):

                    if dx == dy == dz == 0:
                        continue

                    x, y, z = i + dx, j + dy, k + dz

                    if 0 <= x < N_a and 0 <= y < N_a and 0 <= z < N_a: # Check if the neighbor is within the lattice
                        
                        coo_a = np.array([i*a, j*a, k*a])
                        coo_b = np.array([x*a, y*a, z*a])

                        if np.linalg.norm(coo_a - coo_b) >= cutoff:
                            continue

                        h_field[i, j, k] += 0.25 * V_ij(coo_a, coo_b,epsilon=1.0)

                        count += 1

    return h_field

# Function to compute total energy of a configuration with open boundary conditions
def compute_energy(lattice, h_field, L=L, N_a=N_a, cutoff=cutoff):
    # import time
    # t0 = time.time()
    E = 0
    a = L / (N_a-1)  # Lattice constant of the underlying simple cubic lattice (in reduced units) 
    edges = []
    for i, j, k in itertools.product(range(N_a), range(N_a), range(N_a)):
        for dx in range(-int(cutoff // a), int(cutoff // a) + 1):
            for dy in range(-int(cutoff // a), int(cutoff // a) + 1):
                for dz in range(-int(cutoff // a), int(cutoff // a) + 1):

                    if dx == dy == dz == 0:
                        continue

                    x, y, z = i + dx, j + dy, k + dz

                    if (x<0) or (x>=N_a) or (y<0) or (y>=N_a) or (z<0) or (z>=N_a):
                        continue
                        
                    coo_a = np.array([i*a, j*a, k*a])
                    coo_b = np.array([x*a, y*a, z*a])

                    if np.linalg.norm(coo_a - coo_b) >= cutoff:
                        continue

                    # only count each pair once
                    # edge = set([(i, j, k), (x, y, z)])
                    # if edge in edges:
                    #     continue
                    # edges.append(edge)
                    
                    if dy < 0:
                        continue
                    if dy==0 and dx>0:
                        continue
                    if dy==0 and dx==0 and dz>0:
                        continue

                    E += 0.25 * V_ij(coo_a, coo_b) * lattice[i, j, k] * lattice[x, y, z]
    # Add the on-site field contribution to the total energy
    E += np.sum(h_field * lattice)
    # print(f'compute_energy took {time.time()-t0} seconds')
    return E

# Function to compute total energy of a configuration with open boundary conditions
def compute_energy_occ(lattice, L=L, N_a=N_a, cutoff=cutoff):
    # import time
    # t0 = time.time()
    E = 0
    a = L / (N_a-1)  # Lattice constant of the underlying simple cubic lattice (in reduced units) 
    edges = []
    for i, j, k in itertools.product(range(N_a), range(N_a), range(N_a)):
        for dx in range(-int(cutoff // a), int(cutoff // a) + 1):
            for dy in range(-int(cutoff // a), int(cutoff // a) + 1):
                for dz in range(-int(cutoff // a), int(cutoff // a) + 1):

                    if dx == dy == dz == 0:
                        continue

                    x, y, z = i + dx, j + dy, k + dz

                    if (x<0) or (x>=N_a) or (y<0) or (y>=N_a) or (z<0) or (z>=N_a):
                        continue
                        
                    coo_a = np.array([i*a, j*a, k*a])
                    coo_b = np.array([x*a, y*a, z*a])

                    if np.linalg.norm(coo_a - coo_b) >= cutoff:
                        continue

                    
                    if dy < 0:
                        continue
                    if dy==0 and dx>0:
                        continue
                    if dy==0 and dx==0 and dz>0:
                        continue
                
                    E += V_ij(coo_a, coo_b) * lattice[i, j, k] * lattice[x, y, z]

    return E

def compute_energy_spin(lattice, L=L, N_a=N_a, cutoff=cutoff):
    # import time
    # t0 = time.time()
    E = 0
    a = L / (N_a-1)  # Lattice constant of the underlying simple cubic lattice (in reduced units) 
    edges = []
    for i, j, k in itertools.product(range(N_a), range(N_a), range(N_a)):
        for dx in range(-int(cutoff // a), int(cutoff // a) + 1):
            for dy in range(-int(cutoff // a), int(cutoff // a) + 1):
                for dz in range(-int(cutoff // a), int(cutoff // a) + 1):

                    if dx == dy == dz == 0:
                        continue

                    x, y, z = i + dx, j + dy, k + dz

                    if (x<0) or (x>=N_a) or (y<0) or (y>=N_a) or (z<0) or (z>=N_a):
                        continue
                        
                    coo_a = np.array([i*a, j*a, k*a])
                    coo_b = np.array([x*a, y*a, z*a])

                    if np.linalg.norm(coo_a - coo_b) >= cutoff:
                        continue

                    
                    if dy < 0:
                        continue
                    if dy==0 and dx>0:
                        continue
                    if dy==0 and dx==0 and dz>0:
                        continue
                
                    E += V_ij(coo_a, coo_b) * (lattice[i, j, k]+1) * (lattice[x, y, z]+1) * 0.25

    return E

def occ2spin(lattice):
    lattice = np.array(lattice)
    lattice[lattice==0] = -1
    return lattice



In [3]:
lattice_occ = np.random.choice([0, 1], size=(N_a, N_a, N_a))
# print(f'Occupation lattice: \n{lattice_occ}')
lattice_spin = occ2spin(lattice_occ)
# print(f'Spin lattice: \n{lattice_spin}')
# print(lattice_occ, lattice_spin)

h_field = compute_h_field(lattice_spin)
# print(f'Magnetic field: \n{h_field}')


E_spin = compute_energy(lattice_spin, h_field)
E_spin2 = compute_energy_spin(lattice_spin)
E_occ = compute_energy_occ(lattice_occ)
print(f'E_spin: {E_spin}')
print(f'E_occ: {E_occ}')
print(f'E_occ - E_spin: {E_occ - E_spin}')
print(f'E_spin2: {E_spin2}')
print(f'E_occ - E_spin2: {E_occ - E_spin2}')

E_spin: 157.85122068224914
E_occ: 8634.061589693987
E_occ - E_spin: 8476.210369011738
E_spin2: 8634.061589693987
E_occ - E_spin2: 0.0


In [4]:
print(f'h_field: \n{h_field}')

h_field: 
[[[ 6.77220282 11.18174622 10.07349177  9.83601026  9.77899584
    9.77899584  9.83601026 10.07349177 11.18174622  6.77220282]
  [11.18174622 17.4123887  15.91302473 15.59867987 15.52340193
   15.52340193 15.59867987 15.91302473 17.4123887  11.18174622]
  [10.07349177 15.91302473 14.2785395  13.92525997 13.8382425
   13.8382425  13.92525997 14.2785395  15.91302473 10.07349177]
  [ 9.83601026 15.59867987 13.92525997 13.55633923 13.46382567
   13.46382567 13.55633923 13.92525997 15.59867987  9.83601026]
  [ 9.77899584 15.52340193 13.8382425  13.46382567 13.37131212
   13.37131212 13.46382567 13.8382425  15.52340193  9.77899584]
  [ 9.77899584 15.52340193 13.8382425  13.46382567 13.37131212
   13.37131212 13.46382567 13.8382425  15.52340193  9.77899584]
  [ 9.83601026 15.59867987 13.92525997 13.55633923 13.46382567
   13.46382567 13.55633923 13.92525997 15.59867987  9.83601026]
  [10.07349177 15.91302473 14.2785395  13.92525997 13.8382425
   13.8382425  13.92525997 14.2785395  1